<a href="https://colab.research.google.com/github/kAIspeaker/k.A.I_speaker/blob/main/kiosk_AIspeaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
# # Speech Recognition - pyaudio, wave
# !apt remove libav-tools
# !apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
# !pip install pyaudio
# !pip install wave

# STT
!pip install SpeechRecognition

# TTS
!pip install gtts

# 다중 분류 모델
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2

# voice recording on Google Colab
!pip install ffmpeg-python
!pip install pydub

# 일정 시간마다 Time check
!pip install threading

# 형태소 분석
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip install konlpy JPype1-py3 


import pyaudio
import wave

import speech_recognition as sr
import gtts

import torch
from torch import nn
import numpy as np
import gluonnlp as nlp
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from torch.utils.data import Dataset, DataLoader

import os
import re
import pandas as pd
import nltk

import time
import threading

from konlpy.tag import Okt



In [ ]:
from IPython.display import Javascript, HTML, Audio
from google.colab import output
from google.colab.output import eval_js
from base64 import b64decode

import io
from scipy.io.wavfile import read as wav_read
# from scipy.io.wavfile import write as wav_write

import ffmpeg

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = eval_js('record(%d)' % (sec*1000))
  bi = b64decode(s.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=bi)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]
  # print(type(riff)) bytes

  with open('audio.wav','wb') as f:
    f.write(riff)
  return 'audio.wav'


In [ ]:
def speech_to_text():
	record(5)
	r = sr.Recognizer()

	harvard = sr.AudioFile('audio.wav')
	with harvard as source:
		print('고객 응답: ', end='')
		audio = r.record(source)
	recognized_text = r.recognize_google(audio, language='ko-KR')
	print(recognized_text, '\n')

	return recognized_text

In [ ]:
# deal_menulist
def deal_menulist():
	menu_file = pd.read_csv('/gdrive/My Drive/햄버거_메뉴.csv', header=0)
	menu_main = menu_file['햄버거'].dropna()

	cnt = 0
	for menu in menu_file['햄버거']:
		cnt += 1
		if (cnt % 3) == 0:
			print('%10s' % menu)
			print('\n')
			continue
		print('%10s' % menu, '  |  ', end='')
	print('\n')



# deal_orcer
# 각 카테고리에 있는지 확인
# 상세메뉴가 들어온 경우
# 카테고리로 인식되는 경우랑 -> 메뉴판

# 메뉴가 없는 경우(카테고리로도)
# 수량이 안들어왔을 때는 1개라고 인식
# 변경 주문에 대해서는 인식 불가

def deal_order(sentence, okt):
	global basket

	nouns = [noun for noun in okt.phrases(sentence)]
	nums = [num for num in okt.pos(sentence) if num[1] == 'Number']

	menu_file = pd.read_csv('/gdrive/My Drive/햄버거_메뉴.csv', header=0)
	menu_main = menu_file['햄버거'].dropna()
	menu_side = menu_file['사이드'].dropna()
	menu_drink = menu_file['음료'].dropna()
	menu_cate = menu_file['category'].dropna()
	quant = menu_file['수량'].dropna()

	i = 0
	is_correct_order = 0
	for noun in nouns:
		noun = noun.replace(" ", "")
		if (i < len(nums)):
			quantity = nums[i]
			i += 1
		else:
			quantity = 1

		if menu_main.index[menu_main == noun].to_list(): #메뉴가 있으면
			is_correct_order = 1
			print('메인 메뉴: ', noun, ', 수량: {} 개'.format(quantity))
			basket.append(noun)
		elif menu_side.index[menu_side == noun].to_list():
			is_correct_order = 1
			print('사이드 메뉴: ', noun, ', 수량: {} 개'.format(quantity))
			basket.append(noun)
		elif menu_drink.index[menu_drink == noun].to_list():
			is_correct_order = 1
			print('음료: ', noun, ', 수량: {} 개'.format(quantity))
			basket.append(noun)
		elif menu_cate.index[menu_cate == noun].to_list(): #카테고리로 주문
			is_correct_order = 1
			cnt = 0
			for menu in menu_file[noun].dropna():
				cnt += 1
				if (cnt % 3) == 0:
					print('%10s' % menu)
					print('\n')
					continue
				print('%10s' % menu, '  |  ', end='')
			print('\n')
			break

	if is_correct_order == 0:
		print("주문 메뉴를 인식하지 못했습니다. 다시 말씀해주세요.")

	print("현재 장바구니: ", end='')
	count = {}
	for m in basket:
		try: count[m] += 1
		except: count[m] = 1
	print(count)

	return 0


# deal_changing
# 일단 들어온 주문에 메뉴가 있는지 없는지 확인
#	있으면
# -> 변경재료 있는지 없는지 확인
# 	있으면 
#  			 -> 메뉴가 장바구니에 있는지 확인
#  			    있으면 -> 처리
#           없으면 -> 메뉴 장바구니에 담기
# 	없으면 -> 변경하실 재료를 다시 말씀해주세요

# 없으면
# -> 변경하실 메뉴를 말씀해주세요

#  불고기버거에 양상추 추가요 라고 했는데, 장바구니에 불고기버거가 없는 경우
#  불고기버거를 장바구니에 담고, 변경 메뉴판 띄우기

# 추가, 삭제, 수량 변경, 변경(음료, 세트 등), 사이즈 변경

def deal_changing(sentence, okt):
	global basket

	if (basket == []):
		print("주문 내역이 없습니다. 주문 후 변경 해주세요.")
		return(0)

	nouns = [noun for noun in okt.phrases(sentence)]
	nums = [num for num in okt.pos(sentence) if num[1] == 'Number']

	menu_file = pd.read_csv('/gdrive/My Drive/햄버거_메뉴.csv', header=0)
	menu_main = menu_file['햄버거'].dropna()
	menu_side = menu_file['사이드'].dropna()
	menu_drink = menu_file['음료'].dropna()
	menu_change = menu_file['변경'].dropna()

	menu = 0
	ingred = 0

	is_in_menu = 0
	for noun in nouns:
		noun = noun.replace(" ", "")
		# 들어온 주문에서 메뉴 인식
		if (menu_main.index[menu_main == noun].to_list() or
			menu_side.index[menu_side == noun].to_list() or
			menu_drink.index[menu_drink == noun].to_list()):
			is_in_menu = 1
			menu = noun

	# 메뉴 있음
	if is_in_menu != 0:
		ingred = 0
		for noun in nouns:
			if menu_change.index[menu_change == noun.replace(" ", "")].to_list():
				ingred = noun
		change = 0
		for noun in nouns:
			if (noun == '추가'):
				change = '추가'
			elif (noun == '변경'):
				change = '변경'
			elif (noun == '삭제'):
				change = '삭제'
			elif (noun == '사이즈'):
				change = '사이즈'
		if (change == 0):
			change = '변경'

		# 재료 없음
		if (ingred == 0):
			print("변경 메뉴: {}".format(menu))
			print("변경하실 재료를 말씀해주세요.")
			print('[', end='')
			for c in menu_change:
				print(c, end='')
				if (c != menu_change.iloc[-1]) == True:
					print(', ', end='')
			print(']')
		else: # 메뉴, 재료 둘 다 있음
			is_in_basket = 0
			for m in basket:
				if menu == m:
					is_in_basket = 1 # 장바구니에 있음
			# 메뉴가 장바구니에 있음
			if is_in_basket == 1:
				print('변경 종류: {0}, 메뉴: {1}, 변경 재료: {2}'.format(change, menu, ingred))
			else: # 메뉴가 장바구니에 없음
				basket.append(menu)
				print('\'{}\' 메뉴를 장바구니에 담았습니다.'.format(menu))
				print("변경하실 재료를 말씀해주세요.")
				print('[', end='')
				for c in menu_change:
					print(c, end='')
					if (c != menu_change.iloc[-1]) == True:
						print(', ', end='')
				print(']')
	else: # 메뉴 없음
		print("변경하실 메뉴를 다시 말씀해주세요.")
		print("현재 장바구니: ", end='')
		count = {}
		for m in basket:
			try: count[m] += 1
			except: count[m] = 1
		print(count)

	return 0


# deal_pay
# 가장 먼저 장바구니 확인
# 멤버십을 말하더라도 주문 확인부터

def deal_pay():
	global basket
	global is_processing
	global ALARM_FLAG

	if (basket == []):
		print("주문 내역이 없습니다. 주문 후 결제 해주세요.")
		return(0)
	print("주문하신 메뉴가 맞습니까?")
	print('[주문하신 메뉴]')
	count = {}
	for m in basket:
		try: count[m] += 1
		except: count[m] = 1
	print(count)
	print('\n')

	customer_ans = speech_to_text()
	if customer_ans == '네' or customer_ans == '맞아요' or customer_ans == '응':
		ALARM_FLAG = 0
		print_alarm()
	
		print("포인트를 적립하시겠습니까?")
		customer_ans2 = speech_to_text()
		if (customer_ans2 == '네' or customer_ans2 == '할게요' 
					or customer_ans2 == '할게' or customer_ans2 == '응'):
			print("적립 번호를 말씀해주세요.")
			customer_ans3 = speech_to_text()
			print('\'{}\' 번호로 포인트가 적립되었습니다.'.format(customer_ans3))

		print("카드를 삽입해주세요.")
		time.sleep(3)
		print("결제가 완료되었습니다.\n3초 뒤에 초기 화면으로 돌아갑니다.")
		
		is_processing = 0
		basket = []

		for i in range(3,0,-1):
			print(i)
			time.sleep(1)

	else:
		print("주문을 다시 확인해주세요.")


	return 0

# del_recommand
def deal_recommand():
	menu_file = pd.read_csv('/gdrive/My Drive/햄버거_메뉴.csv', header=0)
	recommand = menu_file['recommand'].dropna()
	print("저희 매장의 추천 메뉴입니다.")
	print("[", end='')
	for i in recommand:
		print(i, end='')
		if (i != recommand.iloc[-1]) == True:
			print(', ', end='')
	print("]")

	return 0

In [ ]:
def rule_base(category, text, okt):

	ret = 0
	if category == '주문':
		ret = deal_order(text, okt)
	elif category == '변경':
		ret = deal_changing(text, okt)
	elif category == '결제' or category == '포인트_적립':
		ret = deal_pay()
	elif category == '추천':
		ret = deal_recommand()
	elif category == '메뉴판':
		ret = deal_menulist()
	else:
		print("주문을 인식하지 못하였습니다. 다시 말씀해주세요.")


	return 0

In [ ]:
def print_menu():
  menu_file = pd.read_csv('/gdrive/My Drive/햄버거_메뉴.csv', header=0)
  menu_main = menu_file['햄버거'].dropna()
  menu_side = menu_file['사이드'].dropna()
  menu_drink = menu_file['음료'].dropna()

  printing_line = ["main: ", "side: ", "drink: "]
  menu_list = [menu_main, menu_side, menu_drink]
  for i, menu in enumerate(menu_list):
    print(printing_line[i])
    print('[', end='')
    for i in menu:
      print(i, end='')
      if (i != menu.iloc[-1]) == True:
        print(', ', end='')
    print(']')

  print('\n\n\n')


In [ ]:
def get_basket():
  global basket
  
  print("현재 장바구니: ", end='')
  count = {}
  for m in basket:
    try: count[m] += 1
    except: count[m] = 1
  print(count)

In [ ]:
def print_alarm():
  global ALARM_FLAG
  global timer

  print("\n결제를 원하시면 \'결제\', 변경을 원하시면 \'변경\'이라고 말씀해주세요.\n")

  timer = threading.Timer(20, print_alarm)
  if ALARM_FLAG == 0:
    timer.cancel()
    return
  timer.start()


In [ ]:
class BERTDataset(Dataset):
	def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
				 pad, pair):
		transform = nlp.data.BERTSentenceTransform(bert_tokenizer,vocab=vocab,max_seq_length=max_len, pad=pad, pair=pair)

		self.sentences = [transform([i[sent_idx]]) for i in dataset]
		self.labels = [np.int32(i[label_idx]) for i in dataset]

	def __getitem__(self, i):
		return (self.sentences[i] + (self.labels[i], ))

	def __len__(self):
		return (len(self.labels))

class BERTClassifier(nn.Module):
	def __init__(self,
				 bert,
				 hidden_size = 768,
				 num_classes=11,
				 dr_rate=None,
				 params=None):
		super(BERTClassifier, self).__init__()
		self.bert = bert
		self.dr_rate = dr_rate

		self.classifier = nn.Linear(hidden_size , num_classes)
		if dr_rate:
			self.dropout = nn.Dropout(p=dr_rate)

	def gen_attention_mask(self, token_ids, valid_length):
		attention_mask = torch.zeros_like(token_ids)
		for i, v in enumerate(valid_length):
			attention_mask[i][:v] = 1
		return attention_mask.float()

	def forward(self, token_ids, valid_length, segment_ids):
		attention_mask = self.gen_attention_mask(token_ids, valid_length)

		_, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
		if self.dr_rate:
			out = self.dropout(pooler)
		return self.classifier(out)



In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

def predict(predict_sentence, model, tok, vocab):

	data = [predict_sentence, '0']
	dataset_another = [data]

	another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
	test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)

	model.eval()

	for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
		token_ids = token_ids.long()
		segment_ids = segment_ids.long()
		valid_length= valid_length
		label = label.long()

		out = model(token_ids, valid_length, segment_ids)

		test_eval=[]
		for i in out:
			logits=i
			logits = logits.detach().cpu().numpy()

			if np.argmax(logits) == 0:
				test_eval.append("결제")
			elif np.argmax(logits) == 1:
				test_eval.append("메뉴판")
			elif np.argmax(logits) == 2:
				test_eval.append("변경")
			elif np.argmax(logits) == 3:
				test_eval.append("영수증")
			elif np.argmax(logits) == 4:
				test_eval.append("재고_확인")
			elif np.argmax(logits) == 5:
				test_eval.append("정보_가격")
			elif np.argmax(logits) == 6:
				test_eval.append("정보_메뉴")
			elif np.argmax(logits) == 7:
				test_eval.append("정보_제휴")
			elif np.argmax(logits) == 8:
				test_eval.append("주문")
			elif np.argmax(logits) == 9:
				test_eval.append("추천")
			elif np.argmax(logits) == 10:
				test_eval.append("포인트_적립")

		return (test_eval[0])

In [ ]:
def get_model():
  model = torch.load('/gdrive/My Drive/k_A_I_speaker.pt', map_location=torch.device('cpu'))
  tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
  vocab = tokenizer.vocab_file
  vocab = nlp.vocab.BERTVocab.from_sentencepiece(vocab, padding_token='[PAD]')

  tok = tokenizer.tokenize

  return model, tok, vocab

In [ ]:
def main(model, tok, vocab):
  global basket
  global is_processing
  global ALARM_FLAG

  okt = Okt()

  while True:
    is_processing = 0
    basket = []

    print("주문을 원하시면, '주문' 이라고 말씀해 주세요.\n시각장애인이시면 소지하신 이어폰을 왼쪽 하단에 꽂아주세요.")
    start = speech_to_text()
    print('\n')
    
    if (start.replace(" ", "") == '주문'):
      is_processing = 1
      ALARM_FLAG = 0
      
      while is_processing == 1:
        if basket != [] and ALARM_FLAG == 0: #결제&변경 알림 처리
          ALARM_FLAG = 1
          print_alarm()
        
        print("주문을 말씀해주세요.")
        sentence = speech_to_text()
        
        if (sentence == '종료'):
          print("키오스크 주문이 종료되었습니다.\n")
          break
        elif (sentence == '장바구니'):
          get_basket()
        elif (sentence == '변경'):
          ret = deal_changing(sentence, okt)
        elif (sentence == '결제'):
          ret = deal_pay()
        else:
          category = predict(sentence, model, tok, vocab)
          print("카테고리: {}".format(category))

          rule_base(category, sentence, okt)

        print('\n\n\n')

      ALARM_FLAG = 0
      print_alarm() #결제 완료 및 주문 종료시 결제&변경 알림도 종료

    elif (start == '종료'):
      print("키오스크 주문이 종료되었습니다.")
      break

In [ ]:
if __name__ == "__main__":
	global basket
	global is_processing
	global ALARM_FLAG

	print_menu()
 
	model, tok, vocab = get_model()

	main(model, tok, vocab)
 



main: 
[비프버거, 불고기버거, 더블와규버거, 치킨버거, 새우버거, 아재버거, 싸이버거, 더블불고기버거, 더블치즈버거, 핫크리스피버거, 상하이버거, 치즈버거, 쉬림프버거, 빅맥, 칠리핫버거, 미라클버거, 폴더버거, 징거버거, 더블베이컨버거, 할로윈스테이크버거, 콰트로치즈와퍼]
side: 
[맥치킨, 치즈스틱, 너겟, 양념감자, 코울슬로, 웨지감자, 골든모짜렐라치즈스틱]
drink: 
[콜라, 제로콜라, 사이다, 아메리카노, 쉐이크, 오렌지주스, 탄산수, 우유, 라떼, 물]




주문을 원하시면, '주문' 이라고 말씀해 주세요.
시각장애인이시면 소지하신 이어폰을 왼쪽 하단에 꽂아주세요.
고객 응답: 주문 



주문을 말씀해주세요.
고객 응답: 콰트로치즈와퍼 세트 한 개 주세요 

카테고리: 주문
메인 메뉴:  콰트로치즈와퍼 , 수량: 1 개
현재 장바구니: {'콰트로치즈와퍼': 1}





결제를 원하시면 '결제', 변경을 원하시면 '변경'이라고 말씀해주세요.

주문을 말씀해주세요.
고객 응답: 콰트로 치즈 와퍼 치즈 추가해주세요 

카테고리: 변경
변경 종류: 추가, 메뉴: 콰트로치즈와퍼, 변경 재료: 치즈




주문을 말씀해주세요.
고객 응답: 결제 

주문하신 메뉴가 맞습니까?
[주문하신 메뉴]
{'콰트로치즈와퍼': 1}



결제를 원하시면 '결제', 변경을 원하시면 '변경'이라고 말씀해주세요.

고객 응답: 네 

포인트를 적립하시겠습니까?
고객 응답: 아니요 

카드를 삽입해주세요.
결제가 완료되었습니다.
3초 뒤에 초기 화면으로 돌아갑니다.
3
2
1




주문을 원하시면, '주문' 이라고 말씀해 주세요.
시각장애인이시면 소지하신 이어폰을 왼쪽 하단에 꽂아주세요.
